In [1]:
# Upload a positive and a negative file review from the data provided
from google.colab import files
uploaded = files.upload()

Saving negative.review to negative.review
Saving positive.review to positive.review


In [2]:
import nltk
nltk.download('punkt')
import numpy as np
import pandas as pd

from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from nltk.stem import PorterStemmer 

import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
positive_reviews = BeautifulSoup(open('positive.review').read(), features="html5lib")
positive_reviews = positive_reviews.findAll('review_text')
positive_reviews = [i.contents[0] for i in positive_reviews]

negative_reviews = BeautifulSoup(open('negative.review').read(), features="html5lib")
negative_reviews = negative_reviews.findAll('review_text')
negative_reviews = [i.contents[0] for i in negative_reviews]

In [8]:
data = {'Text': positive_reviews + negative_reviews, 'Sentiment': [1] * len(positive_reviews) + [0] * len(negative_reviews)}
df = pd.DataFrame(data = data)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Sentiment'], test_size=0.3)

In [16]:
ps = PorterStemmer() 

def stemming(s):
    s = s.lower()
    tokens = nltk.tokenize.word_tokenize(s)
    tokens = [t for t in tokens if len(t) >= 3]
    tokens = [t.translate(str.maketrans('', '', string.punctuation)) for t in tokens]
    tokens = [t for t in tokens if not any(i.isdigit() for i in t)]
    tokens = [ps.stem(t) for t in tokens]
    
    return tokens

count_vect_model = CountVectorizer(decode_error='ignore', tokenizer = stemming)
count_vect_model.fit(X_train)

X_train_cnt = count_vect_model.transform(X_train)
X_test_cnt = count_vect_model.transform(X_test)

model = LogisticRegression()
model.fit(X_train_cnt, y_train)

print("Test Accuracy:", round(model.score(X_test_cnt, y_test), 3))

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


Test Accuracy: 0.76


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [12]:
class ValuedWord:
  def __init__(self, word, value):
    self.word = word
    self.value = value

valuedWords = []

threshold = 0.8
for i, j in zip(count_vect_model.get_feature_names_out(), model.coef_[0]):
    if j > threshold or j < -threshold:
        vw = ValuedWord(i, round(j, 3))
        valuedWords.append(vw)

In [13]:
# entry the sentence to test
entrySen = input()

that is a bad product


In [14]:
from nltk.tokenize import word_tokenize
words = word_tokenize(entrySen)
ps = PorterStemmer()

In [15]:
senValue = 0

for vw in valuedWords:
  for word in words:
    if word == vw.word:
      senValue += vw.value

if senValue != 0:
  if senValue > 0:
    print('The sentence is positive!')
  else:
    print('The sentence is negative!')

The sentence is negative!
